In [ ]:
# %%
# -------------------- IMPORTS AND CONFIGURATION --------------------
import os
import glob
import re
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import umap
import plotly.express as px
import seaborn as sns
import colorcet as cc
import multimatch_gaze as mm
from joblib import Parallel, delayed

# --- Configuration ---
# The notebook is in code/analysis/multimatch
# The data is in the root directory relative to 'code'
BASE_DIR = "../../" # Go up two levels from the current script location
DATA_DIR = os.path.join(BASE_DIR, "preprocess")
OUTPUT_DIR = os.path.join(BASE_DIR, "analysis", "multimatch", "output") # New output location

# --- NEW: Path to pre-processed fixations ---
FIXATIONS_DIR = os.path.join(DATA_DIR, "events", "fixations")

# --- NEW: Output file locations ---
METADATA_MANIFEST_FILE = os.path.join(OUTPUT_DIR, "metadata_manifest.csv")
DISTANCE_MATRIX_FILE = os.path.join(OUTPUT_DIR, "distance_matrix_5d.npy") # Main output

# --- Parameters ---
# Screen dimensions for normalization (assuming input coordinates are already 0-1)
SCREEN_DIMS = [1.0, 1.0] 

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Data directory: {os.path.abspath(DATA_DIR)}")
print(f"Fixations directory: {os.path.abspath(FIXATIONS_DIR)}")
print(f"Output will be saved to: {os.path.abspath(OUTPUT_DIR)}")

In [ ]:
# %%
# -------------------- LOAD PRE-COMPUTED SCANPATHS FROM PARQUET (No Caching) --------------------

# --- This function loads a single parquet file and extracts the scanpath ---
def extract_scanpath_from_parquet(path):
    df = pd.read_parquet(path)
    # The required columns are already in the correct format
    scanpath_np = df[['x', 'y', 'duration_ms']].to_numpy()
    
    # Extract proband, stimulus, and binary label code from filename
    filename = os.path.basename(path)
    match = re.match(r"(P\d+)_IMG(\d+)_(\d+).parquet", filename)
    if match:
        proband_id = match.group(1)
        stimulus_id_num = int(match.group(2))
        binary_code = match.group(3)
    else:
        # Fallback if filename format is different
        proband_id, stimulus_id_num, binary_code = 'unknown', 0, 'unknown'
        
    return {
        'proband_id': proband_id,
        'stimulus_id': f'id{stimulus_id_num}', # Recreate the 'idXXX' format
        'binary_code': binary_code,
        'scanpath': scanpath_np,
        'original_file': filename
    }

# --- Main Loading Logic (Always reads from source Parquet files) ---
print(f"Building scanpaths directly from Parquet files in: {FIXATIONS_DIR}...")
files = sorted(glob.glob(os.path.join(FIXATIONS_DIR, '*.parquet')))

# Check if any files were found
if not files:
    raise FileNotFoundError(f"FATAL ERROR: No Parquet files found in the directory '{FIXATIONS_DIR}'. Please check the 'FIXATIONS_DIR' path in your configuration.")

scanpaths_data = []

for f in tqdm(files, desc="Processing Parquet files"):
    try:
        sp = extract_scanpath_from_parquet(f)
        # Basic validation: ensure the scanpath has at least 2 fixations
        if sp and len(sp['scanpath']) > 1:
            scanpaths_data.append(sp)
    except Exception as e:
        print(f"Failed on {f}: {e}")

print(f"\nSuccessfully loaded {len(scanpaths_data)} scanpaths directly from source.")

In [ ]:
# %%
# -------------------- BUILD THE DISTANCE MATRIX & METADATA MANIFEST (PARALLELIZED & 5D) --------------------
import itertools
from joblib import Parallel, delayed

# --- Configuration for Testing ---
STIMULI_TO_PROCESS = []

# --- 1. Filter the data if we are in testing mode ---
if STIMULI_TO_PROCESS:
    print(f"--- RUNNING IN SUBSET MODE FOR STIMULI: {STIMULI_TO_PROCESS} ---")
    # --- FIX: Use the correct key 'original_file' for filtering ---
    scanpaths_data_subset = [sp for sp in scanpaths_data if (match := re.search(r'(id\d+)', sp['original_file'])) and match.group(1) in STIMULI_TO_PROCESS]
    scanpaths_data = scanpaths_data_subset
    METADATA_MANIFEST_FILE = os.path.join(OUTPUT_DIR, "metadata_manifest_TEST.csv")
    DISTANCE_MATRIX_FILE = os.path.join(OUTPUT_DIR, "distance_matrix_5d_TEST.npy")
    print(f"Filtered down to {len(scanpaths_data)} scanpaths for testing.")
else:
    print("--- RUNNING IN FULL MODE ON ALL STIMULI ---")
    METADATA_MANIFEST_FILE = os.path.join(OUTPUT_DIR, "metadata_manifest.csv")
    DISTANCE_MATRIX_FILE = os.path.join(OUTPUT_DIR, "distance_matrix_5d.npy")

# --- 2. Create and save the Metadata Manifest ---
print("\nCreating metadata manifest file...")
metadata_records = []
for i, sp_data in enumerate(scanpaths_data):
    # --- FIX: Use the correct key 'original_file' ---
    file, scanpath = sp_data['original_file'], sp_data['scanpath']
    stim_match = re.search(r'(id\d+)', file)
    proband_match = re.search(r'(proband\d+|p\d+)', file, re.IGNORECASE)
    metadata_records.append({
        'matrix_index': i, 'proband_id': proband_match.group(0) if proband_match else 'unknown_proband',
        'stimulus_id': stim_match.group(1) if stim_match else 'unknown_stimulus',
        'n_fixations': len(scanpath), 'original_file': file
    })
metadata_df = pd.DataFrame(metadata_records)
metadata_df.to_csv(METADATA_MANIFEST_FILE, index=False)
print(f"Saved metadata manifest to: {METADATA_MANIFEST_FILE}")

# --- 3. Define a Helper Function for a Single Comparison ---
def calculate_dissimilarity_vector(scanpath1_np, scanpath2_np):
    scanpath1_df = pd.DataFrame(scanpath1_np, columns=["start_x", "start_y", "duration"])
    scanpath2_df = pd.DataFrame(scanpath2_np, columns=["start_x", "start_y", "duration"])
    similarity_vector_nested = mm.docomparison(scanpath1_df, scanpath2_df, screensize=SCREEN_DIMS)
    similarity_vector = np.array(similarity_vector_nested[0])
    return 1 - similarity_vector

# --- 4. Build or Load the 5D Distance Matrix ---
if os.path.exists(DISTANCE_MATRIX_FILE):
    print(f"\nLoading pre-computed 5D distance matrix from: {DISTANCE_MATRIX_FILE}")
    distance_matrix_5d = np.load(DISTANCE_MATRIX_FILE)
else:
    print(f"\nComputing 5D pairwise distance matrix for {len(scanpaths_data)} scanpaths using all available CPU cores...")
    n = len(scanpaths_data)
    pairs_to_compare = list(itertools.combinations(range(n), 2))

    results = Parallel(n_jobs=-1)(
        delayed(calculate_dissimilarity_vector)(
            scanpaths_data[i]['scanpath'], scanpaths_data[j]['scanpath']
        ) for i, j in tqdm(pairs_to_compare, desc="Comparing Scanpaths")
    )

    distance_matrix_5d = np.zeros((n, n, 5))
    for pair, result_vector in zip(pairs_to_compare, results):
        i, j = pair
        distance_matrix_5d[i, j, :] = result_vector
        distance_matrix_5d[j, i, :] = result_vector

    np.save(DISTANCE_MATRIX_FILE, distance_matrix_5d)
    print(f"Saved 5D distance matrix to: {DISTANCE_MATRIX_FILE}")

# --- 5. Create the 2D Matrix for Downstream Compatibility ---
print("\nCalculating 2D Euclidean norm matrix from the 5D data...")
distance_matrix = np.linalg.norm(distance_matrix_5d, axis=2)
print("   ...done.")

print("\nDistance matrices (5D and 2D) and metadata manifest are ready.")

In [ ]:
# %%
# -------------------- CLEAN THE 5D DISTANCE MATRIX AND SYNC METADATA --------------------

# --- 1. Determine Which Files to Load (Test or Full Run) ---
# We check if a TEST version of the matrix exists. If so, we use the TEST files.
# Otherwise, we assume we are in a full run and use the standard files.

# Check if the STIMULI_TO_PROCESS variable was set in the previous cell.
# This makes the logic more explicit.
try:
    if STIMULI_TO_PROCESS:
        print("--- Operating in SUBSET/TEST mode ---")
        METADATA_TO_LOAD = os.path.join(OUTPUT_DIR, "metadata_manifest_TEST.csv")
        MATRIX_TO_LOAD = os.path.join(OUTPUT_DIR, "distance_matrix_5d_TEST.npy")
    else:
        print("--- Operating in FULL mode ---")
        METADATA_TO_LOAD = os.path.join(OUTPUT_DIR, "metadata_manifest.csv")
        MATRIX_TO_LOAD = os.path.join(OUTPUT_DIR, "multimatch_distance_matrix.npy")
except NameError:
    # If the variable doesn't exist, fall back to the full run files
    print("--- Operating in FULL mode (default) ---")
    METADATA_TO_LOAD = os.path.join(OUTPUT_DIR, "metadata_manifest.csv")
    MATRIX_TO_LOAD = os.path.join(OUTPUT_DIR, "multimatch_distance_matrix.npy")

# --- 2. Load the 5D Matrix AND the Metadata Manifest ---
try:
    print(f"\nLoading metadata from: {METADATA_TO_LOAD}")
    metadata_df = pd.read_csv(METADATA_TO_LOAD)

    print(f"Loading 5D distance matrix from: {MATRIX_TO_LOAD}")
    distance_matrix_5d = np.load(MATRIX_TO_LOAD)

except FileNotFoundError:
    raise FileNotFoundError(
        "FATAL ERROR: Could not find the matrix or metadata file. Please run the 'BUILD THE DISTANCE MATRIX' cell first.")

# --- Initial Sanity Check ---
if distance_matrix_5d.shape[0] != len(metadata_df):
    raise ValueError(
        f"CRITICAL ERROR: Mismatch between loaded matrix ({distance_matrix_5d.shape[0]}) and metadata ({len(metadata_df)}). "
        "Please delete both files and re-run the 'BUILD' cell."
    )

print(f"\nOriginal 5D distance matrix shape: {distance_matrix_5d.shape}")
print(f"Original metadata length: {len(metadata_df)}")

# --- 3. Step 1: Filter out rows/cols that are ONLY np.nan or 0.0 ---
is_zero_or_nan_5d = (distance_matrix_5d == 0) | np.isnan(distance_matrix_5d)
is_invalid_vector_matrix = is_zero_or_nan_5d.all(axis=2)
is_invalid_row_mask = is_invalid_vector_matrix.all(axis=1)
pass1_keep_mask = ~is_invalid_row_mask
pass1_indices = np.where(pass1_keep_mask)[0]

matrix_5d_pass1 = distance_matrix_5d[np.ix_(pass1_indices, pass1_indices)]
metadata_pass1 = metadata_df.iloc[pass1_indices]

print(
    f"\nStep 1: Removed {len(distance_matrix_5d) - len(matrix_5d_pass1)} rows that were entirely composed of zero/NaN vectors.")
print(f"   Matrix shape after Step 1: {matrix_5d_pass1.shape}")

# --- 4. Step 2: From the remainder, filter out any rows/cols that still contain ANY np.nan ---
contains_any_nan_mask = np.isnan(matrix_5d_pass1).any(axis=(1, 2))
pass2_keep_mask = ~contains_any_nan_mask
pass2_local_indices = np.where(pass2_keep_mask)[0]

distance_matrix_5d_clean = matrix_5d_pass1[np.ix_(pass2_local_indices, pass2_local_indices)]
metadata_df_clean = metadata_pass1.iloc[pass2_local_indices].reset_index(drop=True)

print(
    f"\nStep 2: Removed {len(matrix_5d_pass1) - len(distance_matrix_5d_clean)} additional rows that contained remaining NaNs.")

# --- 5. Create the Final 2D Matrix for UMAP ---
print("\nCalculating final 2D Euclidean norm matrix from the cleaned 5D data...")
distance_matrix_clean = np.linalg.norm(distance_matrix_5d_clean, axis=2)
print("   ...done.")

print(f"\nFinal cleaned 5D matrix shape: {distance_matrix_5d_clean.shape}")
print(f"Final cleaned 2D matrix shape: {distance_matrix_clean.shape}")
print(f"Final cleaned metadata length: {len(metadata_df_clean)}")

# Final sanity check
if distance_matrix_clean.shape[0] != len(metadata_df_clean):
    raise ValueError("CRITICAL ERROR: Final matrix and metadata lengths do not match after cleaning!")

print("\nCleaning complete. Ready for UMAP.")

In [ ]:
# %%
# -------------------- DIMENSIONALITY REDUCTION & 3D VISUALIZATION --------------------
print("Running UMAP on the pre-computed and cleaned distance matrix...")

# --- 3D UMAP Embedding ---
reducer_3d = umap.UMAP(
    n_components=3,
    n_neighbors=200,
    metric='precomputed',  # Crucial!
    random_state=42
)
# Use the CLEANED distance matrix
embedding_3d = reducer_3d.fit_transform(distance_matrix_clean)

# --- Prepare data for plotting from the CLEANED metadata ---
emb_df_3d = pd.DataFrame(embedding_3d, columns=['u0', 'u1', 'u2'])

# Assign columns directly from the clean metadata DataFrame
emb_df_3d['file'] = metadata_df_clean['original_file']
emb_df_3d['stimulus'] = metadata_df_clean['stimulus_id']
emb_df_3d['n_fixations'] = metadata_df_clean['n_fixations']

unique_stimuli = emb_df_3d['stimulus'].unique()
glasbey_palette = cc.glasbey_light
color_map = {stimulus: glasbey_palette[i % len(glasbey_palette)] for i, stimulus in enumerate(unique_stimuli)}
print(f"Generated a distinct color map for {len(unique_stimuli)} stimuli.")

# Create the 3D scatter plot
fig_3d = px.scatter_3d(
    emb_df_3d,
    x='u0', y='u1', z='u2',
    hover_name='file',
    color='stimulus',
    color_discrete_map=color_map,
    size='n_fixations',
    title='3D UMAP Projection (MultiMatch Comparison)'
)
fig_3d.update_traces(marker=dict(opacity=0.8))

out_html_3d = os.path.join(OUTPUT_DIR, 'umap_scanpaths_3d_multimatch.html')
fig_3d.write_html(out_html_3d)
print(f"Saved interactive 3D UMAP to: {out_html_3d}")

fig_3d.show()

In [ ]:
# %%
# -------------------- 2D VISUALIZATION --------------------
print("Generating 2D interactive plot...")

# --- 2D UMAP Embedding ---
reducer_2d = umap.UMAP(
    n_components=2,
    n_neighbors=200,
    min_dist=0.8,
    metric='precomputed',  # Use the distance matrix
    random_state=42
)
# Use the CLEANED distance matrix
embedding_2d = reducer_2d.fit_transform(distance_matrix_clean)

# --- Prepare data for plotting from the CLEANED metadata ---
emb_df_2d = pd.DataFrame(embedding_2d, columns=['u0', 'u1'])

# Assign columns directly from the clean metadata DataFrame
emb_df_2d['file'] = metadata_df_clean['original_file']
emb_df_2d['stimulus'] = metadata_df_clean['stimulus_id']
emb_df_2d['n_fixations'] = metadata_df_clean['n_fixations']

unique_stimuli = emb_df_2d['stimulus'].unique()
glasbey_palette = cc.glasbey_light
color_map = {stimulus: glasbey_palette[i % len(glasbey_palette)] for i, stimulus in enumerate(unique_stimuli)}
print(f"Generated a distinct color map for {len(unique_stimuli)} stimuli.")

# Create the 2D scatter plot
fig_2d = px.scatter(
    emb_df_2d,
    x='u0', y='u1',
    hover_name='file',
    color='stimulus',
    color_discrete_map=color_map,
    size='n_fixations',
    title='2D UMAP Projection (MultiMatch Comparison)'
)
fig_2d.update_traces(marker=dict(opacity=0.8))

out_html_2d = os.path.join(OUTPUT_DIR, 'umap_scanpaths_2d_multimatch.html')
fig_2d.write_html(out_html_2d)
print(f"Saved interactive 2D UMAP to: {out_html_2d}")

fig_2d.show()

In [ ]:
# %%
# -------------------- VISUALIZE THE 5 DIMENSIONAL UMAP EMBEDDINGS (with Glasbey Colors) --------------------

print("Generating 5 separate scatter plots, one for each dimensional UMAP embedding...")

# --- 1. Prepare Data for Plotting ---
try:
    _ = embeddings
    _ = metadata_df_clean
except NameError:
    raise NameError("FATAL ERROR: The 'embeddings' or 'metadata_df_clean' variables were not found. Please run the previous cell first.")

dimensions = ['Shape', 'Length', 'Direction', 'Position', 'Duration']
stimulus_labels = metadata_df_clean['stimulus_id']

unique_stimuli = stimulus_labels.unique()
glasbey_palette = cc.glasbey_light
color_map = {stimulus: glasbey_palette[i % len(glasbey_palette)] for i, stimulus in enumerate(unique_stimuli)}
print(f"Generated a distinct Glasbey color map for {len(unique_stimuli)} stimuli.")

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, dim_name in enumerate(dimensions):
    ax = axes[i]
    embedding = embeddings[dim_name]

    sns.scatterplot(
        x=embedding[:, 0],
        y=embedding[:, 1],
        hue=stimulus_labels,
        palette=color_map,
        ax=ax,
        s=15,
        alpha=0.6,
        linewidth=0,
        legend='auto'
    )
    
    ax.set_title(f'UMAP Embedding of "{dim_name}" Dimension')
    ax.set_xlabel('UMAP Dimension 1')
    ax.set_ylabel('UMAP Dimension 2')
    ax.legend().set_visible(False)

axes[-1].set_visible(False)
fig.suptitle("2D UMAP Embeddings for Each of the 5 MultiMatch Dimensions", fontsize=20, y=1.03)
plt.tight_layout()

output_embeddings_plot_path = os.path.join(OUTPUT_DIR, 'dimensional_embeddings_scatter.png')
plt.savefig(output_embeddings_plot_path, dpi=300)
print(f"\nSaved the 5 embedding plots to: {output_embeddings_plot_path}")

plt.show()

In [ ]:
# %%
# -------------------- ANALYZE STIMULUS CHARACTERISTICS FROM DISTANCE MATRIX --------------------

print("Analyzing stimulus characteristics from the cleaned distance matrix...")

# --- Safety Check: Ensure required variables from previous cells exist ---
try:
    _ = distance_matrix_clean
    _ = metadata_df_clean
except NameError:
    raise NameError("FATAL ERROR: Required variables ('distance_matrix_clean', 'metadata_df_clean') not found. Please run the cleaning cell first.")

# --- 1. Calculate Intra- and Inter-Stimulus Distances ---
unique_stimuli = metadata_df_clean['stimulus_id'].unique()
results = []

for stimulus_id in tqdm(unique_stimuli, desc="Analyzing Stimuli"):
    
    # Get the indices for scanpaths belonging to the current stimulus
    indices_within = metadata_df_clean[metadata_df_clean['stimulus_id'] == stimulus_id].index
    
    # Get the indices for scanpaths NOT belonging to the current stimulus
    indices_outside = metadata_df_clean[metadata_df_clean['stimulus_id'] != stimulus_id].index
    
    # --- Calculate Intra-Stimulus Distance (Variability) ---
    avg_intra_distance = np.nan # Default to NaN if calculation is not possible
    # We need at least 2 participants for a stimulus to measure its internal variability
    if len(indices_within) > 1:
        # Create a sub-matrix containing only distances between participants for this stimulus
        sub_matrix_within = distance_matrix_clean[np.ix_(indices_within, indices_within)]
        # Calculate the mean of the upper triangle of this matrix (to ignore the diagonal 0s and duplicates)
        avg_intra_distance = np.mean(sub_matrix_within[np.triu_indices_from(sub_matrix_within, k=1)])
        
    # --- Calculate Inter-Stimulus Distance (Uniqueness) ---
    avg_inter_distance = np.nan # Default to NaN if calculation is not possible
    if len(indices_outside) > 0:
        # Create a sub-matrix of distances from this stimulus's participants to all others
        sub_matrix_outside = distance_matrix_clean[np.ix_(indices_within, indices_outside)]
        # The average of this entire sub-matrix is the average distance to all other stimuli
        avg_inter_distance = np.mean(sub_matrix_outside)
        
    results.append({
        'stimulus_id': stimulus_id,
        'num_probands': len(indices_within),
        'avg_intra_distance': avg_intra_distance, # High value = High Variability
        'avg_inter_distance': avg_inter_distance  # High value = High Uniqueness
    })

# Convert the results into a DataFrame for easy sorting and display
stimulus_analysis_df = pd.DataFrame(results)

# --- 2. Find and Display the Results ---

print("\n" + "="*80)
print("Stimulus Gaze Behavior Analysis")
print("="*80)

# --- Variability / Constancy ---
variability_sorted = stimulus_analysis_df.sort_values(by='avg_intra_distance', ascending=False, na_position='last')
most_variable = variability_sorted.iloc[0]
most_constant = variability_sorted.iloc[-1]

print("\n--- Variability (Intra-Stimulus Distance) ---\n")
print(f"Most VARIABLE Stimulus:      '{most_variable['stimulus_id']}'")
print(f"  - Average distance between its probands: {most_variable['avg_intra_distance']:.4f}")
print("  - Interpretation: Gaze behavior for this stimulus was the most inconsistent across different people.\n")

print(f"Most CONSTANT Stimulus:      '{most_constant['stimulus_id']}'")
print(f"  - Average distance between its probands: {most_constant['avg_intra_distance']:.4f}")
print("  - Interpretation: Gaze behavior for this stimulus was the most consistent and predictable across people.\n")

# --- Uniqueness / Averageness ---
uniqueness_sorted = stimulus_analysis_df.sort_values(by='avg_inter_distance', ascending=False, na_position='last')
most_unique = uniqueness_sorted.iloc[0]
most_average = uniqueness_sorted.iloc[-1]

print("\n--- Uniqueness (Inter-Stimulus Distance) ---\n")
print(f"Most UNIQUE ('Distant') Stimulus: '{most_unique['stimulus_id']}'")
print(f"  - Average distance to all other stimuli: {most_unique['avg_inter_distance']:.4f}")
print("  - Interpretation: The gaze behavior for this stimulus was the most distinct from the typical gaze behavior seen across the entire dataset.\n")

print(f"Most AVERAGE ('Typical') Stimulus: '{most_average['stimulus_id']}'")
print(f"  - Average distance to all other stimuli: {most_average['avg_inter_distance']:.4f}")
print("  - Interpretation: The gaze behavior for this stimulus was the most representative of the overall average gaze behavior.\n")


# --- 3. Display the Full Data Table for Context ---
print("\n" + "="*80)
print("Full Analysis Data Table")
print("="*80)
display(stimulus_analysis_df.sort_values(by='avg_intra_distance', ascending=False).reset_index(drop=True))